In [4]:
import json
import os

# Load JSON data from database files
print("Current Working Directory (CWD):", os.getcwd())
with open('../assets/employees.json') as f:
    employees = json.load(f)
with open('../assets/categories.json') as f:
    categories = json.load(f)
with open('../assets/images.json') as f:
    images = json.load(f)
with open('../assets/keycardentries.json') as f:
    keycard_entries = json.load(f)


print(employees[0].keys())
print(categories[0].keys())
print(images[0].keys())
print(keycard_entries[0].keys())

Current Working Directory (CWD): c:\Users\demen\Documents\FWC_Project\tests
dict_keys(['EmployeeId', 'FirstName', 'LastName', 'WorkEmail', 'WorkTitle', 'StartDate', 'BirthDate', 'ProfilePictureId', 'IsActive', 'KeyCardId'])
dict_keys(['categoryId', 'categoryName'])
dict_keys(['ImageId', 'ImageName', 'ImageExtension', 'ImageData', 'ImageCategoryId'])
dict_keys(['EntryId', 'EntryDateTime', 'KeyCardId', 'ScannerId', 'EntryCategoryId', 'SecurityImageId'])


In [5]:
# Search function to find employees by name
def search_employees_by_name(name_query):
    name_query = name_query.lower().strip()
    results = [e for e in employees if name_query
                in (e['FirstName'].lower() + ' ' + e['LastName'].lower())]
    return results

print(search_employees_by_name("Jean"))
print(search_employees_by_name("Jean Mar"))
print(search_employees_by_name("S"))

[{'EmployeeId': 1, 'FirstName': 'Jean', 'LastName': 'Martinez', 'WorkEmail': 'jean_martinez@myfwc.com', 'WorkTitle': 'OPS Associate Research Scientist', 'StartDate': '2022-05-16', 'BirthDate': '1994-11-04', 'ProfilePictureId': 101, 'IsActive': 1, 'KeyCardId': '11111111-1111-1111-1111-111111111111'}]
[{'EmployeeId': 1, 'FirstName': 'Jean', 'LastName': 'Martinez', 'WorkEmail': 'jean_martinez@myfwc.com', 'WorkTitle': 'OPS Associate Research Scientist', 'StartDate': '2022-05-16', 'BirthDate': '1994-11-04', 'ProfilePictureId': 101, 'IsActive': 1, 'KeyCardId': '11111111-1111-1111-1111-111111111111'}]
[{'EmployeeId': 4, 'FirstName': 'Omar', 'LastName': 'Sanchez', 'WorkEmail': 'omar_sanchez@myfwc.com', 'WorkTitle': 'Biological Scientist IV', 'StartDate': '2002-06-01', 'BirthDate': '1970-07-12', 'ProfilePictureId': 104, 'IsActive': 1, 'KeyCardId': '44444444-4444-4444-4444-444444444444'}, {'EmployeeId': 5, 'FirstName': 'Mina', 'LastName': 'Kowalski', 'WorkEmail': 'mina_kowalski@myfwc.com', 'Work

In [6]:
def retrieve_employee_keycard_entries(employee_keyCardId):
    entries = [e for e in keycard_entries if e['KeyCardId'] == employee_keyCardId]
    return entries

print(retrieve_employee_keycard_entries('11111111-1111-1111-1111-111111111111'))

[{'EntryId': 'e0001001-0001-0001-0001-000000000001', 'EntryDateTime': '2025-09-16T08:30:00Z', 'KeyCardId': '11111111-1111-1111-1111-111111111111', 'ScannerId': 'fd001001-fd01-fd01-fd01-fd0000000001', 'EntryCategoryId': 54, 'SecurityImageId': 201}, {'EntryId': 'e0001001-0001-0001-0001-000000000002', 'EntryDateTime': '2025-09-16T12:00:00Z', 'KeyCardId': '11111111-1111-1111-1111-111111111111', 'ScannerId': 'fd001001-fd01-fd01-fd01-fd0000000001', 'EntryCategoryId': 55, 'SecurityImageId': 202}, {'EntryId': 'e0001001-0001-0001-0001-000000000003', 'EntryDateTime': '2025-09-16T12:45:00Z', 'KeyCardId': '11111111-1111-1111-1111-111111111111', 'ScannerId': 'fd001001-fd01-fd01-fd01-fd0000000001', 'EntryCategoryId': 54, 'SecurityImageId': 203}, {'EntryId': 'e0001001-0001-0001-0001-000000000004', 'EntryDateTime': '2025-09-16T17:15:00Z', 'KeyCardId': '11111111-1111-1111-1111-111111111111', 'ScannerId': 'fd001001-fd01-fd01-fd01-fd0000000001', 'EntryCategoryId': 55, 'SecurityImageId': 204}]


In [7]:
from py_linq import Enumerable

employees_enum = Enumerable(employees)
entries_enum = Enumerable(keycard_entries)
images_enum = Enumerable(images)
categories_enum = Enumerable(categories)

joined_data = employees_enum.join(entries_enum, 
                                  lambda emp: emp['KeyCardId'], 
                                  lambda key: key['KeyCardId'], 
                                  lambda result: {'Employee': result[0], 'KeyCardEntry': result[1]})
print(joined_data)

joined_data_images = joined_data.join(images_enum,
                                      lambda left: left['KeyCardEntry']['SecurityImageId'],
                                      lambda image: image['ImageId'],
                                      lambda result: {'Employee': result[0]['Employee'],
                                                      'KeyCardEntry': result[0]['KeyCardEntry'],
                                                      'Image': result[1]})
print(joined_data_images)

final_data = joined_data_images.join(categories_enum,
                                     lambda left: left['Image']['ImageCategoryId'],
                                     lambda category: category['categoryId'],
                                     lambda result: {'Employee': result[0]['Employee'],
                                                    'KeyCardEntry': result[0]['KeyCardEntry'],
                                                    'Image': result[0]['Image'],
                                                    'Category': result[1]})
print(final_data)

                                                        

[{'Employee': {'EmployeeId': 1, 'FirstName': 'Jean', 'LastName': 'Martinez', 'WorkEmail': 'jean_martinez@myfwc.com', 'WorkTitle': 'OPS Associate Research Scientist', 'StartDate': '2022-05-16', 'BirthDate': '1994-11-04', 'ProfilePictureId': 101, 'IsActive': 1, 'KeyCardId': '11111111-1111-1111-1111-111111111111'}, 'KeyCardEntry': {'EntryId': 'e0001001-0001-0001-0001-000000000001', 'EntryDateTime': '2025-09-16T08:30:00Z', 'KeyCardId': '11111111-1111-1111-1111-111111111111', 'ScannerId': 'fd001001-fd01-fd01-fd01-fd0000000001', 'EntryCategoryId': 54, 'SecurityImageId': 201}}, {'Employee': {'EmployeeId': 1, 'FirstName': 'Jean', 'LastName': 'Martinez', 'WorkEmail': 'jean_martinez@myfwc.com', 'WorkTitle': 'OPS Associate Research Scientist', 'StartDate': '2022-05-16', 'BirthDate': '1994-11-04', 'ProfilePictureId': 101, 'IsActive': 1, 'KeyCardId': '11111111-1111-1111-1111-111111111111'}, 'KeyCardEntry': {'EntryId': 'e0001001-0001-0001-0001-000000000002', 'EntryDateTime': '2025-09-16T12:00:00Z', 

In [8]:
def retrieve_information_by_name(name_query):
    name_query = name_query.lower().strip()
    results = (final_data
               .where(lambda entry: name_query in (entry['Employee']['FirstName'].lower() + ' ' + entry['Employee']['LastName'].lower())))
    return results

print(retrieve_information_by_name("Omar"))
    

[{'Employee': {'EmployeeId': 4, 'FirstName': 'Omar', 'LastName': 'Sanchez', 'WorkEmail': 'omar_sanchez@myfwc.com', 'WorkTitle': 'Biological Scientist IV', 'StartDate': '2002-06-01', 'BirthDate': '1970-07-12', 'ProfilePictureId': 104, 'IsActive': 1, 'KeyCardId': '44444444-4444-4444-4444-444444444444'}, 'KeyCardEntry': {'EntryId': 'e0001004-0004-0004-0004-000000000001', 'EntryDateTime': '2025-09-16T08:00:00Z', 'KeyCardId': '44444444-4444-4444-4444-444444444444', 'ScannerId': 'fd001002-fd02-fd02-fd02-fd0000000002', 'EntryCategoryId': 54, 'SecurityImageId': 211}, 'Image': {'ImageId': 211, 'ImageName': 'SecurityCam_Sanchez_20250916_0800', 'ImageExtension': 'jpg', 'ImageData': '/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCab', 'ImageCategoryId': 16}, 'Category': {'categoryId': 16, 'categoryName': 'Security Camera Footage'}}, {'Employee': {'EmployeeId': 4, 'FirstName': 'Omar', 'Las

In [9]:
from datetime import datetime
DATE_FORMAT = "%Y-%m-%dT%H:%M:%SZ"
def retrieve_information_by_date_range(start_date, end_date):

    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, DATE_FORMAT)
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, DATE_FORMAT)

    if start_date and not end_date or not start_date and end_date:
        end_date = start_date

    results = (final_data
               .where(lambda entry: start_date <= datetime.strptime(entry['KeyCardEntry']['EntryDateTime'],DATE_FORMAT) <= end_date))
    return results

print(retrieve_information_by_date_range(datetime.strptime("2025-09-16T08:00:00Z", DATE_FORMAT), datetime.strptime("2025-09-16T10:00:00Z", DATE_FORMAT)))

[{'Employee': {'EmployeeId': 1, 'FirstName': 'Jean', 'LastName': 'Martinez', 'WorkEmail': 'jean_martinez@myfwc.com', 'WorkTitle': 'OPS Associate Research Scientist', 'StartDate': '2022-05-16', 'BirthDate': '1994-11-04', 'ProfilePictureId': 101, 'IsActive': 1, 'KeyCardId': '11111111-1111-1111-1111-111111111111'}, 'KeyCardEntry': {'EntryId': 'e0001001-0001-0001-0001-000000000001', 'EntryDateTime': '2025-09-16T08:30:00Z', 'KeyCardId': '11111111-1111-1111-1111-111111111111', 'ScannerId': 'fd001001-fd01-fd01-fd01-fd0000000001', 'EntryCategoryId': 54, 'SecurityImageId': 201}, 'Image': {'ImageId': 201, 'ImageName': 'SecurityCam_Martinez_20250916_0830', 'ImageExtension': 'jpg', 'ImageData': '/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCal', 'ImageCategoryId': 16}, 'Category': {'categoryId': 16, 'categoryName': 'Security Camera Footage'}}, {'Employee': {'EmployeeId': 2, 'FirstName': 

In [10]:
def retrieve_information_by_both(name_query, start_date, end_date):
    name_query = name_query.lower().strip()
    DATE_FORMAT = "%Y-%m-%dT%H:%M:%SZ"
    results = (final_data
               .where(lambda entry: name_query in (entry['Employee']['FirstName'].lower() + ' ' + entry['Employee']['LastName'].lower()) and
                                  start_date <= datetime.strptime(entry['KeyCardEntry']['EntryDateTime'],DATE_FORMAT) <= end_date))
    return results
print(retrieve_information_by_both("Omar", datetime.strptime("2025-09-16T08:00:00Z", DATE_FORMAT), datetime.strptime("2025-09-16T10:00:00Z", DATE_FORMAT)))

[{'Employee': {'EmployeeId': 4, 'FirstName': 'Omar', 'LastName': 'Sanchez', 'WorkEmail': 'omar_sanchez@myfwc.com', 'WorkTitle': 'Biological Scientist IV', 'StartDate': '2002-06-01', 'BirthDate': '1970-07-12', 'ProfilePictureId': 104, 'IsActive': 1, 'KeyCardId': '44444444-4444-4444-4444-444444444444'}, 'KeyCardEntry': {'EntryId': 'e0001004-0004-0004-0004-000000000001', 'EntryDateTime': '2025-09-16T08:00:00Z', 'KeyCardId': '44444444-4444-4444-4444-444444444444', 'ScannerId': 'fd001002-fd02-fd02-fd02-fd0000000002', 'EntryCategoryId': 54, 'SecurityImageId': 211}, 'Image': {'ImageId': 211, 'ImageName': 'SecurityCam_Sanchez_20250916_0800', 'ImageExtension': 'jpg', 'ImageData': '/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCab', 'ImageCategoryId': 16}, 'Category': {'categoryId': 16, 'categoryName': 'Security Camera Footage'}}]


In [11]:
def retrieve_information(name_query = None, start_date = None, end_date = None):
    results = final_data
    if name_query:
        if start_date and end_date:
            results = retrieve_information_by_both(name_query, start_date, end_date)
        if start_date and not end_date:
            results = retrieve_information_by_both(name_query, start_date, datetime.max)
        if not start_date and end_date:
            results = retrieve_information_by_both(name_query, datetime.min, end_date)
        if not start_date and not end_date:
            results = retrieve_information_by_name(name_query)
    else:
        if start_date and end_date:
            results = retrieve_information_by_date_range(start_date, end_date)
        if start_date and not end_date:
            results = retrieve_information_by_date_range(start_date, datetime.max)
        if not start_date and end_date:
            results = retrieve_information_by_date_range(datetime.min, end_date)
    return results.to_list()

print(retrieve_information(name_query="Omar", start_date=datetime.strptime("2025-09-16T08:00:00Z", DATE_FORMAT), end_date=datetime.strptime("2025-09-16T10:00:00Z", DATE_FORMAT)))

[{'Employee': {'EmployeeId': 4, 'FirstName': 'Omar', 'LastName': 'Sanchez', 'WorkEmail': 'omar_sanchez@myfwc.com', 'WorkTitle': 'Biological Scientist IV', 'StartDate': '2002-06-01', 'BirthDate': '1970-07-12', 'ProfilePictureId': 104, 'IsActive': 1, 'KeyCardId': '44444444-4444-4444-4444-444444444444'}, 'KeyCardEntry': {'EntryId': 'e0001004-0004-0004-0004-000000000001', 'EntryDateTime': '2025-09-16T08:00:00Z', 'KeyCardId': '44444444-4444-4444-4444-444444444444', 'ScannerId': 'fd001002-fd02-fd02-fd02-fd0000000002', 'EntryCategoryId': 54, 'SecurityImageId': 211}, 'Image': {'ImageId': 211, 'ImageName': 'SecurityCam_Sanchez_20250916_0800', 'ImageExtension': 'jpg', 'ImageData': '/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQYHBwYIChAKCgkJChQODwwQFxQYGBcUFhYaHSUfGhsjHBYWICwgIyYnKSopGR8tMC0oMCUoKSj/2wBDAQcHBwoIChMKChMoGhYaKCgoKCgoKCgoKCab', 'ImageCategoryId': 16}, 'Category': {'categoryId': 16, 'categoryName': 'Security Camera Footage'}}]


In [12]:
import pandas as pd

final_data_list = final_data.to_list()
df = pd.json_normalize(
    final_data_list,
    sep='_',
    max_level=2 
)
df.columns = [
    col.split('_', 1)[-1] 
    for col in df.columns
]
display(df)  # Display the first few rows of the DataFrame

,EmployeeId,FirstName,LastName,WorkEmail,WorkTitle,StartDate,BirthDate,ProfilePictureId,IsActive,KeyCardId,...,ScannerId,EntryCategoryId,SecurityImageId,ImageId,ImageName,ImageExtension,ImageData,ImageCategoryId,categoryId,categoryName
0,1,Jean,Martinez,jean_martinez@myfwc.com,OPS Associate Research Scientist,2022-05-16,1994-11-04,101,1,11111111-1111-1111-1111-111111111111,...,fd001001-fd01-fd01-fd01-fd0000000001,54,201,201,SecurityCam_Martinez_20250916_0830,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
1,1,Jean,Martinez,jean_martinez@myfwc.com,OPS Associate Research Scientist,2022-05-16,1994-11-04,101,1,11111111-1111-1111-1111-111111111111,...,fd001001-fd01-fd01-fd01-fd0000000001,55,202,202,SecurityCam_Martinez_20250916_1200,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
2,1,Jean,Martinez,jean_martinez@myfwc.com,OPS Associate Research Scientist,2022-05-16,1994-11-04,101,1,11111111-1111-1111-1111-111111111111,...,fd001001-fd01-fd01-fd01-fd0000000001,54,203,203,SecurityCam_Martinez_20250916_1245,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
3,1,Jean,Martinez,jean_martinez@myfwc.com,OPS Associate Research Scientist,2022-05-16,1994-11-04,101,1,11111111-1111-1111-1111-111111111111,...,fd001001-fd01-fd01-fd01-fd0000000001,55,204,204,SecurityCam_Martinez_20250916_1715,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
4,2,Alex,Nguyen,alex_nguyen@myfwc.com,Scientific Programmer,2023-01-10,1997-08-21,102,1,22222222-2222-2222-2222-222222222222,...,fd001002-fd02-fd02-fd02-fd0000000002,54,205,205,SecurityCam_Nguyen_20250916_0845,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
5,2,Alex,Nguyen,alex_nguyen@myfwc.com,Scientific Programmer,2023-01-10,1997-08-21,102,1,22222222-2222-2222-2222-222222222222,...,fd001002-fd02-fd02-fd02-fd0000000002,55,206,206,SecurityCam_Nguyen_20250916_1730,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
6,3,Rina,Patel,rina_patel@myfwc.com,Data Administration Analyst,2021-07-12,1996-09-18,103,1,33333333-3333-3333-3333-333333333333,...,fd001001-fd01-fd01-fd01-fd0000000001,54,207,207,SecurityCam_Patel_20250916_0900,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
7,3,Rina,Patel,rina_patel@myfwc.com,Data Administration Analyst,2021-07-12,1996-09-18,103,1,33333333-3333-3333-3333-333333333333,...,fd001001-fd01-fd01-fd01-fd0000000001,55,208,208,SecurityCam_Patel_20250916_1200,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
8,3,Rina,Patel,rina_patel@myfwc.com,Data Administration Analyst,2021-07-12,1996-09-18,103,1,33333333-3333-3333-3333-333333333333,...,fd001001-fd01-fd01-fd01-fd0000000001,54,209,209,SecurityCam_Patel_20250916_1300,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
9,3,Rina,Patel,rina_patel@myfwc.com,Data Administration Analyst,2021-07-12,1996-09-18,103,1,33333333-3333-3333-3333-333333333333,...,fd001001-fd01-fd01-fd01-fd0000000001,55,210,210,SecurityCam_Patel_20250916_1800,jpg,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAYEBQYFBAYGBQ...,16,16,Security Camera Footage
